##  Connect to MySQL

In [4]:
import peewee
import pymysql

db = peewee.MySQLDatabase('db_test',
                          user='root',
                          host='localhost',
                          password='root5438',
                          port=3306)

###  selecting the data I want to see from cluster result

In [5]:
import pymysql.cursors
import codecs

try :
    with db.cursor() as cursor:
            # Read a single record
            sql = "SELECT avg(AverageSpeed), max(AverageSpeed), min(AverageSpeed) FROM group4data_trial_complete where VEHICLE_ID in (%s) "%str(list(df_cluster['Cluster 4'])[:]).replace('[', '').replace(']', '')
            cursor.execute(sql)
            result_statistic = cursor.fetchall()
            #print(result_statistic)
finally:
    db.close()

NameError: name 'df_cluster' is not defined

In [ ]:
# SQL snytax
group by CarBrand_Ultimate
group by OwnerGender
avg(AverageSpeed), max(AverageSpeed), min(AverageSpeed)
CarBrand_Ultimate, count(*)


result_statistic


result
result1

### save into MySQL

In [3]:
try :
    with db.cursor() as cursor:
            # Read a single record
            sql = "SELECT * FROM group4data_trial_complete where VEHICLE_ID in (%s) "%str(list(df_cluster['Cluster 0'])[:]).replace('[', '').replace(']', '')
            cursor.execute(sql)
            result_statistic = cursor.fetchall()
            sql_createTableAs = "create table cluster_2_4 as SELECT * FROM group4data_trial_complete where VEHICLE_ID in (%s)"%str(list(df_cluster['Cluster 4'])[:]).replace('[', '').replace(']', '')
            cursor.execute(sql_createTableAs)
            #print(result_statistic)
finally:
    db.close()

NameError: name 'df_cluster' is not defined

#### for statistic result and turn into csv

In [111]:
col_1 = ['gender', 'count']
df_result_sta2 = pd.DataFrame(list(result_statistic), columns = col_1)

In [112]:
df_result_sta2

gender  count
0      F    564
1      M    482
2      U    107

#### (ALL) result to csv

In [67]:
columns = ['VEHICLE_ID', 'DetectedCarCategory', 'IdentityType', 'CarBrand', 'RegisteredCarCategory' , 'CarFactorized_YM', 'OwnerGender', 'CarCounty', 'TotalKM2017', 'TotalDay2017', 'AverageDistance', 'DelayedPayBillTimes', 'WeekdayRatio', 'WeekendRatio', 'NorthRatio', 'MidRatio', 'SouthRatio', 'DayPassingRatio', 'NightPassingRatio', 'AverageSpeed', 'OverSpeedRatio', 'bigger350KM', 'ACL_days', 'CarBrand_Ultimate']
df_result1 = pd.DataFrame(list(result1), columns = columns)

In [6]:
#df_result1

In [113]:
df_result_sta2.to_csv('C://Users//Administrator//Documents//code//data//DTP_D4//cluster_2//cluster4_sta2.csv',columns = col_1 , index=False , header = True,sep=',',encoding='utf-8')

## Cluster from dummies (category and numeric) 

In [2]:
import pandas as pd
df_new = pd.read_csv('C://Users//Administrator//Documents//code//data//DTP_D4//altered_g4data_1.csv', na_filter=True, encoding = 'utf-8')

In [3]:
df_brand = pd.get_dummies(df_new.CarBrand_Ultimate)
df_gender = pd.get_dummies(df_new.OwnerGender)
train = pd.concat([df_new.VEHICLE_ID, df_new.AverageSpeed, df_brand, df_gender], axis = 1)

In [4]:
train.set_index('VEHICLE_ID', inplace=True)

In [7]:
#train.head()

###  calculate the distance between instances, for the similarity

In [6]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

In [7]:
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance


def instance_distance(dataframe, name, distance_measures, n):
    # We want a data frame to store the output
    # distance_measures is a list of the distance measures you want to compute (see below)
    # n is how many "most similar" to report
    distances = pd.DataFrame()
    
    # Find the location of the instance we are looking for
    instance_location = np.where(dataframe.index == name)[0][0]

    # Go through all distance measures we care about
    for distance_measure in distance_measures:
        # Find all pairwise distances
        current_distances = distance.squareform(distance.pdist(dataframe, distance_measure))
        # Get the closest n elements for the instance we care about
        most_similar = np.argsort(current_distances[:, instance_location])[0:n]
        # Append results (a new column to the dataframe with the name of the measure)
        distances[distance_measure] = list(zip(dataframe.index[most_similar], current_distances[most_similar, instance_location]))
    return distances

In [8]:
instance_distance(train, 4586024, ['euclidean'], 6)

euclidean
0                   (4586024, 0.0)
1  (10315036, 0.28999999999999915)
2    (4288745, 0.4100000000000037)
3    (1734752, 0.5600000000000023)
4     (1086632, 1.422146265332789)
5    (6062278, 1.4545102268461358)

In [9]:
instance_distance(train, 4586024, ['euclidean', 'cityblock', 'cosine', 'jaccard'], 10)

euclidean                        cityblock  \
0                   (4586024, 0.0)                   (4586024, 0.0)   
1  (10315036, 0.28999999999999915)  (10315036, 0.28999999999999915)   
2    (4288745, 0.4100000000000037)    (4288745, 0.4100000000000037)   
3    (1734752, 0.5600000000000023)    (1734752, 0.5600000000000023)   
4     (1086632, 1.422146265332789)    (1086632, 2.1499999999999986)   
5    (6062278, 1.4545102268461358)    (10551524, 2.240000000000002)   
6   (10948962, 1.4902684321960251)    (9450602, 2.3299999999999983)   
7    (5453418, 1.7149052451957798)    (6062278, 2.3399999999999963)   
8    (1700054, 1.7436742815101696)    (7718717, 2.4299999999999997)   
9    (6092298, 1.9482299658921196)    (10948962, 2.469999999999999)   

                               cosine                         jaccard  
0                      (4586024, 0.0)                  (4586024, 0.0)  
1    (10315036, 8.50123449414042e-09)    (858350, 0.3333333333333333)  
2   (4288745, 1.7065477808664298e-08)  (14515828, 0.3333333333333333)  
3    (1734752, 3.200835485550613e-08)   (4288745, 0.3333333333333333)  
4  (10551524, 4.6427034949836354e-07)  (14141651, 0.3333333333333333)  
5    (9450602, 5.007847987403835e-07)   (4619772, 0.3333333333333333)  
6    (7718717, 5.428379826755148e-07)  (13686605, 0.3333333333333333)  
7     (5340387, 6.50031796833872e-07)   (2781543, 0.3333333333333333)  
8    (2018651, 7.608932420799519e-07)   (3011490, 0.3333333333333333)  
9    (6269444, 8.063978809147443e-07)  (15437157, 0.3333333333333333)

###  cluster using KMeans

In [10]:
from sklearn.cluster import KMeans


k_clusters = 6

## Fit clusters like in our previous models/transformations/standarization 
## (e.g. Logistic, Vectorization,...)

model = KMeans(k_clusters)
model.fit(train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [11]:
print ("Records in our dataset (rows): ", len(train.index))
print ("Then we predict one cluster per record, which means length of: ", len(model.predict(train)) )

('Records in our dataset (rows): ', 5000)
('Then we predict one cluster per record, which means length of: ', 5000)


In [12]:
train.index

Int64Index([ 3949830,  3149488,  2799093,  9405464,  4586024,  3743998,
             4277144,  1547464,  6195186,  6297615,
            ...
            12853414,  5120973, 10777490,  5012544, 13219720,  4760251,
             2555735,  6697167,  8199634,   366919],
           dtype='int64', name=u'VEHICLE_ID', length=5000)

In [15]:
clusters = model.predict(train)

clusters

array([0, 3, 2, ..., 5, 4, 5])

In [13]:
pd.DataFrame(list(zip(train.index,model.predict(train))), columns=['VEHICLE_ID','Cluster_predicted']) [0:10]

VEHICLE_ID  Cluster_predicted
0     3949830                  0
1     3149488                  3
2     2799093                  2
3     9405464                  4
4     4586024                  2
5     3743998                  3
6     4277144                  3
7     1547464                  5
8     6195186                  0
9     6297615                  5

#### making cluster into a column

In [16]:
cluster_listing = {}
for cluster in range(k_clusters):
    cluster_listing['Cluster ' + str(cluster)] = [''] * 5000
    where_in_cluster = np.where(clusters == cluster)[0]
    cluster_listing['Cluster ' + str(cluster)][0:len(where_in_cluster)] = train.index[where_in_cluster]

# Print clusters
pd.DataFrame(cluster_listing).loc[0:np.max(np.bincount(clusters)) - 1,:]

Cluster 0 Cluster 1 Cluster 2 Cluster 3 Cluster 4 Cluster 5
0      3949830   3126157   2799093   3149488   9405464   1547464
1      6195186   3895132   4586024   3743998  11018678   6297615
2     13826146   3685187   6815483   4277144   6497990  13708723
3      8393181   7424312   1115955  14668019   5201962   4724001
4     11684385   1265701  11730395   1018111   8986234  12905907
5     14603901   4203634  11134795   7647131  14395592  12915793
6      2253172   5413065   3083640  10436843   7215622   8934868
7     10286420   6347041  10383873   5017341  15093580  14997480
8     14243958  15427525   8400635     78603  12990616   6168568
9     14843953   6630208   3619852   4584896   4197074    283049
10     2315752   3392555  13390150  13700194    380598   4312323
11    14952610   4362410   5192468  10191295  10431434    861284
12     9575605   3954358   3123030   9696236  12560136   5063048
13     4815478   8126554   2637369   3125497   4117263   2521025
14    10730529  11881975   4545188  10329757   9219449   7683038
15     2245425   5577518   2475766   2369197   1332785  11612623
16     1156441   1749718   5161133   9011919   7389625    669667
17      979347   2489537  10032233   4215943  11580853  14702191
18     3794144   2044935   8697809   6927173   9851036   2481672
19    12891729   6523950   5260162   2399848  13548119   9800164
20      889670   3566495   2193126   9274038  14249968   1342920
21     5511258  15347591    171073   4113262  13347361  13770109
22    12645927   2568804   4221108  10146411  12576713  10708201
23      228347   4846713   9449502   9672061    430772   6171103
24      669272   8685633   3541719   3860736  13000167   5590412
25     2637734   4753326   9421268  13283073   4610824  12637398
26     3556939   3876963   3805183   4971282  12978622  12521514
27     3549635   6392701   3174621   2270569   4011049  10019858
28     3538019   5535385   2263127   3391691  10252570  12944992
29    11363591   5446272   8887369   1640709  13420669   8407504
...        ...       ...       ...       ...       ...       ...
1379                                 2804133                    
1380                                 9563556                    
1381                                 5145573                    
1382                                11288927                    
1383                                 3082322                    
1384                                 3512885                    
1385                                 6807664                    
1386                                 4975886                    
1387                                 5002323                    
1388                                 5030650                    
1389                                12365401                    
1390                                 4375541                    
1391                                  676785                    
1392                                 9152914                    
1393                                 2440919                    
1394                                 1720496                    
1395                                 2455244                    
1396                                  840619                    
1397                                 3501960                    
1398                                12182883                    
1399                                  989694                    
1400                                 1396391                    
1401                                14262653                    
1402                                 7531940                    
1403                                12009795                    
1404                                11884469                    
1405                                 2310253                    
1406                                  850689                    
1407                                11663169                    
1408                                 288

In [17]:
df_cluster = pd.DataFrame(cluster_listing).loc[0:np.max(np.bincount(clusters)) - 1,:]

In [18]:
(str(x) for x in list(df_cluster['Cluster 0']))

<generator object <genexpr> at 0x0000000007A50B40>

In [34]:
str(list(df_cluster['Cluster 0'])[x]) for x in range(len(list(df_cluster['Cluster 0'])))

SyntaxError: invalid syntax (<ipython-input-34-3e35616d4392>, line 1)

In [8]:
#str(list(df_cluster['Cluster 4'])[:]).replace('[', '').replace(']', '')